In [1]:
!ls

aiops_pipeline_artifact
aiops_pipeline_metadata
data
main.py
notebook
README.md
requirements.txt
Untitled.ipynb
venv


In [2]:
!ls data

data.csv


In [3]:
import os

In [7]:
csv_file_path = os.path.join("data","data.csv")

In [5]:
os.path.exists(csv_file_path)

True

In [6]:
csv_file_path

'data\\data.csv'

In [7]:
import csv

In [12]:
import tensorflow as tf

In [8]:
with open(csv_file_path) as csv_file:
    reader =csv.DictReader(csv_file,delimiter=",",quotechar='"')
    for row in reader:
        print(row)
        break

OrderedDict([('pickup_community_area', '60'), ('fare', '27.05'), ('trip_start_month', '10'), ('trip_start_hour', '2'), ('trip_start_day', '3'), ('trip_start_timestamp', '1380593700'), ('pickup_latitude', '41.836150155'), ('pickup_longitude', '-87.648787952'), ('dropoff_latitude', ''), ('dropoff_longitude', ''), ('trip_miles', '12.6'), ('pickup_census_tract', ''), ('dropoff_census_tract', ''), ('payment_type', 'Cash'), ('company', 'Taxi Affiliation Services'), ('trip_seconds', '1380'), ('dropoff_community_area', ''), ('tips', '0.0')])


In [9]:
row

OrderedDict([('pickup_community_area', '60'),
             ('fare', '27.05'),
             ('trip_start_month', '10'),
             ('trip_start_hour', '2'),
             ('trip_start_day', '3'),
             ('trip_start_timestamp', '1380593700'),
             ('pickup_latitude', '41.836150155'),
             ('pickup_longitude', '-87.648787952'),
             ('dropoff_latitude', ''),
             ('dropoff_longitude', ''),
             ('trip_miles', '12.6'),
             ('pickup_census_tract', ''),
             ('dropoff_census_tract', ''),
             ('payment_type', 'Cash'),
             ('company', 'Taxi Affiliation Services'),
             ('trip_seconds', '1380'),
             ('dropoff_community_area', ''),
             ('tips', '0.0')])

In [10]:
row['trip_start_month']

'10'

In [14]:
tf_record_dir = os.path.join(os.getcwd(),"tf_record_files")
tf_record_dir

'D:\\aiops_project\\aiops_project\\tf_record_files'

In [24]:
## to create the directory
os.makedirs(tf_record_dir,exist_ok=True)

In [17]:
tf_record_file_name =  os.path.join(tf_record_dir,'data.tfrecord')

In [19]:
## creates emty tf file
tf_record_writer = tf.io.TFRecordWriter(tf_record_file_name)

In [50]:
def _bytes_feature(value):
    value=value.encode()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [51]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [56]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

In [26]:
_bytes_feature(value='Akash')

value: "Akash"

In [27]:
_int64_feature(value=45)

value: 45

In [28]:
_float_feature(value=56.6)

value: 56.599998474121094

In [30]:
tf.train.Feature(bytes_list=_bytes_feature("Akash"))

bytes_list {
  value: "Akash"
}

In [33]:
{"name": tf.train.Feature(bytes_list=_bytes_feature("Akash"))}

{'name': bytes_list {
   value: "Akash"
 }}

In [35]:
tf.train.Features(feature={"name": tf.train.Feature(bytes_list=_bytes_feature("Akash Borgalli"))})

feature {
  key: "name"
  value {
    bytes_list {
      value: "Akash Borgalli"
    }
  }
}

In [44]:
row_example=tf.train.Example(features=tf.train.Features(feature={"name": tf.train.Feature(bytes_list=_bytes_feature("Akash Borgalli")),
                                                    "age": tf.train.Feature(int64_list=_int64_feature(24))
                                                    }))

In [47]:
row_example

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 24
      }
    }
  }
  feature {
    key: "name"
    value {
      bytes_list {
        value: "Akash Borgalli"
      }
    }
  }
}

In [49]:
## To push the row it in tf_record_files
row_example.SerializeToString()

b'\n*\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x18\n\x1a\n\x04name\x12\x12\n\x10\n\x0eAkash Borgalli'

In [57]:
with open(csv_file_path) as csv_file:
    reader = csv.DictReader(csv_file,delimiter=",",quotechar='"')
    for row in reader:
        ## creating a feature
        feature={
           "pickup_community_area": _bytes_feature(row['pickup_community_area']),
            "trip_start_month":_int64_feature(int(row['trip_start_month'])),
            "fare":_float_feature(float(row["fare"]))
           
        }
        break

In [58]:
feature

{'pickup_community_area': bytes_list {
   value: "60"
 },
 'trip_start_month': int64_list {
   value: 10
 },
 'fare': float_list {
   value: 27.049999237060547
 }}

In [60]:
with open(csv_file_path) as csv_file:
    reader = csv.DictReader(csv_file,delimiter=",",quotechar='"')
    for row in reader:
        ## creating a feature
        feature={
           "pickup_community_area": _bytes_feature(row['pickup_community_area']),
            "trip_start_month":_int64_feature(int(row['trip_start_month'])),
            "fare":_float_feature(float(row["fare"]))
           
        }
        ## pass it to features
        features = tf.train.Features(feature=feature)
        ## now we will give to example
        example = tf.train.Example(features=features)
        ## push it to tfrecordfile
        tf_record_writer.write(example.SerializeToString())
## after writing will close the file        
tf_record_writer.close()

In [61]:
from tfx.components import CsvExampleGen
csv_example_gen = CsvExampleGen(input_base=os.path.join(os.getcwd(),"data"))
csv_example_gen

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x000002071C6110B8>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x000002071C6114A8>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

## To create Parquet file from csv file

In [5]:
import os
import pandas as pd
parquet_dataset_dir = os.path.join(os.getcwd(),"parquet_dir")

os.makedirs(parquet_dataset_dir,exist_ok=True)

In [3]:
parquet_file_name="data.parquet"
parquet_file_path  = os.path.join(parquet_dataset_dir,parquet_file_name)
parquet_file_path

'D:\\aiops_project\\aiops_project\\parquet_dir\\data.parquet'

In [8]:
pd.read_csv(csv_file_path).to_parquet(parquet_file_path,index=None)